# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 22 2022 11:19AM,240179,10,0085795259,ISDIN Corporation,Released
1,Jun 22 2022 11:19AM,240179,10,0085795260,ISDIN Corporation,Released
2,Jun 22 2022 11:19AM,240179,10,0085795281,ISDIN Corporation,Released
3,Jun 22 2022 11:19AM,240179,10,0085795286,ISDIN Corporation,Released
4,Jun 22 2022 11:19AM,240179,10,0085795275,ISDIN Corporation,Released
5,Jun 22 2022 11:19AM,240179,10,0085795276,ISDIN Corporation,Released
6,Jun 22 2022 11:19AM,240179,10,0085795285,ISDIN Corporation,Released
7,Jun 22 2022 11:19AM,240179,10,0085795293,ISDIN Corporation,Released
8,Jun 22 2022 10:58AM,240177,20,ACI-8178355,"ACI Healthcare USA, Inc.",Released
9,Jun 22 2022 10:49AM,240175,10,8174581,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,240173,Released,7
37,240174,Released,1
38,240175,Released,6
39,240177,Released,1
40,240179,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240173,NaN,NaN,7.0
240174,NaN,NaN,1.0
240175,NaN,NaN,6.0
240177,NaN,NaN,1.0
240179,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240081,0.0,35.0,2.0
240085,0.0,8.0,4.0
240086,0.0,15.0,2.0
240089,0.0,0.0,1.0
240092,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240081,0,35,2
240085,0,8,4
240086,0,15,2
240089,0,0,1
240092,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240081,0,35,2
1,240085,0,8,4
2,240086,0,15,2
3,240089,0,0,1
4,240092,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240081,,35,2
1,240085,,8,4
2,240086,,15,2
3,240089,,,1
4,240092,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 22 2022 11:19AM,240179,10,ISDIN Corporation
8,Jun 22 2022 10:58AM,240177,20,"ACI Healthcare USA, Inc."
9,Jun 22 2022 10:49AM,240175,10,Eywa Pharma Inc.
15,Jun 22 2022 10:37AM,240174,19,"GCH Granules USA, Inc."
16,Jun 22 2022 10:32AM,240172,10,ISDIN Corporation
54,Jun 22 2022 10:31AM,240173,10,ISDIN Corporation
61,Jun 22 2022 10:23AM,240169,15,"Carwin Pharmaceutical Associates, LLC"
63,Jun 22 2022 10:22AM,240166,15,"Brookfield Pharmaceuticals, LLC"
64,Jun 22 2022 10:09AM,240157,10,Bio-PRF
91,Jun 22 2022 10:09AM,240164,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 22 2022 11:19AM,240179,10,ISDIN Corporation,,,8
1,Jun 22 2022 10:58AM,240177,20,"ACI Healthcare USA, Inc.",,,1
2,Jun 22 2022 10:49AM,240175,10,Eywa Pharma Inc.,,,6
3,Jun 22 2022 10:37AM,240174,19,"GCH Granules USA, Inc.",,,1
4,Jun 22 2022 10:32AM,240172,10,ISDIN Corporation,,,38
5,Jun 22 2022 10:31AM,240173,10,ISDIN Corporation,,,7
6,Jun 22 2022 10:23AM,240169,15,"Carwin Pharmaceutical Associates, LLC",,,2
7,Jun 22 2022 10:22AM,240166,15,"Brookfield Pharmaceuticals, LLC",,,1
8,Jun 22 2022 10:09AM,240157,10,Bio-PRF,,12,15
9,Jun 22 2022 10:09AM,240164,15,"Person & Covey, Inc.",,1,26


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 11:19AM,240179,10,ISDIN Corporation,8,,
1,Jun 22 2022 10:58AM,240177,20,"ACI Healthcare USA, Inc.",1,,
2,Jun 22 2022 10:49AM,240175,10,Eywa Pharma Inc.,6,,
3,Jun 22 2022 10:37AM,240174,19,"GCH Granules USA, Inc.",1,,
4,Jun 22 2022 10:32AM,240172,10,ISDIN Corporation,38,,
5,Jun 22 2022 10:31AM,240173,10,ISDIN Corporation,7,,
6,Jun 22 2022 10:23AM,240169,15,"Carwin Pharmaceutical Associates, LLC",2,,
7,Jun 22 2022 10:22AM,240166,15,"Brookfield Pharmaceuticals, LLC",1,,
8,Jun 22 2022 10:09AM,240157,10,Bio-PRF,15,12,
9,Jun 22 2022 10:09AM,240164,15,"Person & Covey, Inc.",26,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 11:19AM,240179,10,ISDIN Corporation,8,,
1,Jun 22 2022 10:58AM,240177,20,"ACI Healthcare USA, Inc.",1,,
2,Jun 22 2022 10:49AM,240175,10,Eywa Pharma Inc.,6,,
3,Jun 22 2022 10:37AM,240174,19,"GCH Granules USA, Inc.",1,,
4,Jun 22 2022 10:32AM,240172,10,ISDIN Corporation,38,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 11:19AM,240179,10,ISDIN Corporation,8.0,NaN,NaN
1,Jun 22 2022 10:58AM,240177,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
2,Jun 22 2022 10:49AM,240175,10,Eywa Pharma Inc.,6.0,NaN,NaN
3,Jun 22 2022 10:37AM,240174,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,Jun 22 2022 10:32AM,240172,10,ISDIN Corporation,38.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 11:19AM,240179,10,ISDIN Corporation,8.0,0.0,0.0
1,Jun 22 2022 10:58AM,240177,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
2,Jun 22 2022 10:49AM,240175,10,Eywa Pharma Inc.,6.0,0.0,0.0
3,Jun 22 2022 10:37AM,240174,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,Jun 22 2022 10:32AM,240172,10,ISDIN Corporation,38.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2161443,88.0,19.0,0.0
102,240089,1.0,0.0,0.0
12,240156,1.0,0.0,0.0
15,1200769,99.0,1.0,0.0
16,480275,1.0,5.0,1.0
19,1921088,10.0,7.0,0.0
20,1200587,10.0,58.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2161443,88.0,19.0,0.0
1,102,240089,1.0,0.0,0.0
2,12,240156,1.0,0.0,0.0
3,15,1200769,99.0,1.0,0.0
4,16,480275,1.0,5.0,1.0
5,19,1921088,10.0,7.0,0.0
6,20,1200587,10.0,58.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,88.0,19.0,0.0
1,102,1.0,0.0,0.0
2,12,1.0,0.0,0.0
3,15,99.0,1.0,0.0
4,16,1.0,5.0,1.0
5,19,10.0,7.0,0.0
6,20,10.0,58.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,88.0
1,102,Released,1.0
2,12,Released,1.0
3,15,Released,99.0
4,16,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Executing,19.0,0.0,0.0,1.0,5.0,7.0,58.0
Released,88.0,1.0,1.0,99.0,1.0,10.0,10.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,19.0,0.0,0.0,1.0,5.0,7.0,58.0
2,Released,88.0,1.0,1.0,99.0,1.0,10.0,10.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,19.0,0.0,0.0,1.0,5.0,7.0,58.0
2,Released,88.0,1.0,1.0,99.0,1.0,10.0,10.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()